# **Analytical solutions for solute transport in porous media**

________________
In this Jupyter Notebook, we explore the **analytical solutions for solute transport in porous media**. We begin with a one-dimensional case involving the continuous injection of a contaminant, without decay or adsorption, to understand the basic transport behavior. Next, we introduce first-order decay to represent different contaminants and analyze its effect on transport. Finally, we extend the study to both one- and two-dimensional cases, considering continuous and instantaneous injection scenarios.

Lastly, we delve into the **principle of superposition**, applied to both continuous and pulse injections.

**References**
- Ogata, A., & Banks, R. B. (1961). A solution of the differential equation of longitudinal dispersion in porous media: fluid movement in earth materials. US Government Printing Office.
- Bear, J., & Cheng, A. H. D. (2010). Modeling groundwater flow and contaminant transport (Vol. 23, p. 834). Dordrecht: Springer.
- Bear, J. (2012). Hydraulics of groundwater. Courier Corporation.

_______________

In [15]:
#-- Check and install required packages if not already installed 
import sys
import subprocess

def if_require(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        
#-- Install packages only if they aren't already installed 
if_require("ipywidgets")
if_require("openpyxl")
if_require("xlrd")
if_require("numpy")
if_require("scipy")
if_require("os")

#-- Import the rest of libraries
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import scipy as sp
import os

from IPython.display import display, clear_output, HTML
from ipywidgets import Layout, Output
from tkinter import Tk, filedialog

import warnings
warnings.filterwarnings("ignore")

#-- LaTeX configuration, if you have it installed
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Computer Modern Roman"
# })

#-- Widgets parameters configuration
param_widgets = { 
                 'readout_format': ".2f", 
                 'style': {'description_width': '300px'}, 
                 'layout': widgets.Layout(width='600px')
                 }  

## **1. Analytical solutions for solute transport in porous media, with no decay**

### **1.1. One-dimensional continuous injection**

In this section of the notebook we present the analytical 1D solution for continuous solute transport in porous media without decay, following the classical formulation of Ogata & Banks (1961). Using the hydrodynamic dispersion coefficient and the dimensionless Péclet number, it evaluates solute concentrations as a function of space and time under purely advective–dispersive transport. The equation includes two terms: one representing advective transport and another accounting for dispersion; the notebook explores when the second term becomes negligible depending on dispersivity, providing insight into advection-dominated versus dispersion-dominated regimes.

The analytical solution for solute transport in a 1D continuous injection is the following,


$C(x, t) = \frac{C_{0}}{2}\left[ \operatorname{erfc}\!\left( \frac{x - vt}{\sqrt{4Dt}}\right) + \operatorname{exp}\!\left(\frac{vx}{D}\right) \operatorname{erfc}\!\left( \frac{x + vt}{\sqrt{4Dt}}\right)\right]$

if $\frac{vx}{D} > 100$, we can approximate the solution as $C(x, t) = \frac{C_{0}}{2}\left[ \operatorname{erfc}\!\left( \frac{x - vt}{\sqrt{4Dt}}\right)\right]$

In [2]:
def dispersion(alpha: float, v: float) -> float:
    """Calculates the hydrodynamic dispersion coefficient.

    Parameters
    ----------
    alpha : float
        Dispersivity [L]
    v : float
        Groundwater velocity [L/T]

    Returns
    -------
    float
        Dispersion coefficient [L²/T]
    """
    
    D = alpha * v
    return D

#-------------------------------------------------------#
# Dimensionless parameters: Péclet number, Eta and Xi   #
#-------------------------------------------------------#
def peclet_parameter(x: float, v: int, D: float) -> float:
    """Calculates the Péclet number.

    The Péclet number is a dimensionless parameter that compares
    advective transport to dispersive transport in porous media:
    - High Pe (≫1): advection dominates
    - Low Pe (≪1): dispersion dominates
    
    Parameters
    ----------
    x : float
        Position along the x axis [L]
    v : float
        velocity [L/T]
    D : float
        Dispersion coefficient [L²/T]

    Returns
    -------
    float
        Péclet value [-]
    """
    
    Pe = (v*x)/D
    
    return Pe

def eta_parameter(x: float, v: float, D: float) -> float:
    """Calculates Eta value (Ogata & Banks, 1961).
    
    Eta parameter is the inverse of the Péclet number:
    - Low Eta values: advection-dominated transport
    - High Eta values: dispersion-dominated transport

    Parameters
    ----------
    x : float
        Position along the x axis [L]
    v : float
        Groundwater velocity [L/T]
    D : float
        Dispersion coefficient [L²/T]

    Returns
    -------
    float
        Eta value [-]
    """
    
    Pe = peclet_parameter(x, v, D)
    Eta = 1/Pe
    
    return Eta

def xi_parameter(x: float, v: float, t: float) -> float:
    """Calculates Xi value (Ogata & Banks, 1961). 
    
    Xi is a dimensionless travel-time parameter that gives information about the ratio
    of advective travel distance to actual observation distance:    
    - If Xi < 1, the advective front did not reach the observation point yet
    - If Xi = 0, the advective front just reached the observation point
    - If Xi > 1, the advective front had enough time to arrive to the observation point

    Parameters
    ----------
    x : float
        Position along the x axis [L]
    v : float
        Groundwater velocity [L/T]
    t : float
        Time 

    Returns
    -------
    float
        Xi value [-]
    """
    
    Xi = (v*t)/x
    
    return Xi

def eta_xi_comparison(eta: float, xi: float | np.ndarray) -> np.ndarray:
    """Calculate the normalized concentration C/C0 using the Ogata–Banks solution.

    Parameters
    ----------
    eta : float
        Eta vaues [-]
    xi : float or np.ndarray
        Xi values [-]

    Returns
    -------
    np.ndarray
        Normalized concentration C/C0 [-]
    """
    
    #-- Ogata & Banks (1961)
    term1 = sp.special.erfc( (1 - xi) / (2*(np.sqrt(xi * eta))))
    term2_1 = np.exp(1/eta)
    term2_2 = sp.special.erfc((1 + xi) / (2*(np.sqrt(xi * eta))))
    
    C = (1/2) * (term1 + (term2_1 * term2_2))
    
    return C

#-------------------------------------------------------#
# One-dimensional continuous injection equation         #
#-------------------------------------------------------#
def oneD_continous_injection(Ci: float, x: float | np.ndarray, v: float, t: float | np.ndarray, 
                             alpha: float, adv: bool) -> np.ndarray:
    """Calculates the concentration in a one-dimensional continous injection,
    with no adsorption or decay (Ogata & Banks, 1961).

    Parameters
    ----------
    Ci : float
        Initial concentration [M/V]
    x : float
        Position along the x axis [L]
    v : float
        Groundwater velocity [L/T]
    t : float
        Time
    alpha : float
        Dispersivity [L]
    adv: Boolean
        If False, the second term of the equation will be considered; 
        If True, the second term of the equation will be neglected.

    Returns
    -------
    np.ndarray
        Concentration [M/V]
    """
    #-- Dispersion
    D = dispersion(alpha, v)
    
    #-- Dimensionless transport parameters
    Pe = peclet_parameter(x, v, D)
    Eta = eta_parameter(x, v, D)
    Xi = xi_parameter(x, v, t)
    
    #-- Transport equation 
    term1 = sp.special.erfc((x - (v*t))/(2*(np.sqrt(D*t))))
    term2_1 = np.exp(Pe)
    term2_2 = sp.special.erfc((x + (v*t)) / (2*(np.sqrt(D*t))))
       
    # #-- The equation written in terms of dimensionless parameters. 
    # Ogata & Banks (1961), Eq. 13b (Dimensionless form)
    # term1 = sp.special.erfc((1 - Xi) / (2*(np.sqrt(Xi * Eta))))
    # term2_1 = np.exp(1/Eta)
    # term2_2 = sp.special.erfc((1 + Xi) / (2*(np.sqrt(Xi * Eta))))
            
    if adv == False:
        C = (Ci/2) * (term1 + (term2_1 * term2_2))
    else:
        C = (Ci/2) * (term1)
        
    return C

By varying the values of dispersivity with the slider, we can see when the second term of the equation can be neglected. Remember that changing the dispersivity values directly changes the hydrodynamic dispersion coefficient and, therefore, the Péclet number:

$Pe = \frac{vx}{D}$

In [3]:
def find_parameters_v1(alpha):   
        t_values = [50, 100, 150]
        x_values = [50, 100, 150]
     
        c_space = [oneD_continous_injection(1, np.arange(0, 201), 1, t, alpha, False) for t in t_values]
        c_space_adv = [oneD_continous_injection(1, np.arange(0, 201), 1, t, alpha, True) for t in t_values]
        c_time = [oneD_continous_injection(1, x, 1, np.arange(0, 201), alpha, False) for x in x_values]
        c_time_adv = [oneD_continous_injection(1, x, 1, np.arange(0, 201), alpha, True) for x in x_values]
        
        #-- Plotting the results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        #--space
        ax1.plot(np.arange(0, 201), c_space[0], c="cornflowerblue", label="C(x, 50)")
        ax1.plot(np.arange(0, 201), c_space[1], c="limegreen", label="C(x, 100)")
        ax1.plot(np.arange(0, 201), c_space[2], c="tomato", label="C(0, 150)")
        ax1.plot(np.arange(0, 201), c_space_adv[0], ':', c="cornflowerblue", label="C(x, 50)—2nd term neglected", linewidth=3)
        ax1.plot(np.arange(0, 201), c_space_adv[1], ':', c="limegreen", label="C(x, 100)—2nd term neglected", linewidth=3)
        ax1.plot(np.arange(0, 201), c_space_adv[2], ':', c="tomato", label="C(0, 150)—2nd term neglected", linewidth=3)
        ax1.set_xlim(0, 201)
        ax1.set_ylim(0, 1)
        ax1.set_xlabel(r'x')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        
        #--time
        ax2.plot(np.arange(0, 201), c_time[0], c="cornflowerblue", label="C(50, t)")
        ax2.plot(np.arange(0, 201), c_time[1], c="limegreen", label="C(100, t)")
        ax2.plot(np.arange(0, 201), c_time[2], c="tomato", label="C(150, t)")
        ax2.plot(np.arange(0, 201), c_time_adv[0], ':', c="cornflowerblue", label="C(50, t)—2nd term neglected", linewidth=3)
        ax2.plot(np.arange(0, 201), c_time_adv[1], ':', c="limegreen", label="C(100, t)—2nd term neglected", linewidth=3)
        ax2.plot(np.arange(0, 201), c_time_adv[2], ':', c="tomato", label="C(150, t)—2nd term neglected", linewidth=3)
        ax2.set_xlim(0, 201)
        ax2.set_ylim(0, 1)
        ax2.set_xlabel(r't')
        ax2.set_ylabel('Concentration')
        ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax2.get_position()
        ax2.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        
        fig.tight_layout()
        plt.show()

sliders = [
        widgets.FloatSlider(value=1, min=0.3, max=10, step=0.1, description='Dispersivity [m]', **param_widgets)
]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v1, {
        'alpha': sliders[0]
        })

display(ui, out)

Output()

Next, we can check when the contaminant first reaches an observation point located 125 metres from the source, and determine the maximum concentration there.

Interactive widgets allow users to dynamically adjust parameters such as dispersivity, initial concentration, time, and the groundwater velocity to visualize their impact on concentration profiles. The notebook generates breakthrough curves and spatial concentration distributions, compares full and simplified analytical solutions, and examines contaminant arrival at an observation point. 

In [4]:
def find_parameters_v2(Ci, t, alpha):   
       
        c_space = oneD_continous_injection(Ci, np.arange(0, 201), 1, t, alpha, False)
        
        #-- Plotting the results
        fig, ax1 = plt.subplots(figsize=(10, 3.5))
        
        #--space
        ax1.plot(np.arange(0, 201), c_space, c="cornflowerblue", label="Concentration")
        ax1.set_xlim(0, 201)
        ax1.set_ylim(0, 101)
        ax1.set_xlabel(r'x')
        ax1.set_ylabel('Concentration')
        ax1.vlines(x=125, ymin=0, ymax=100, linestyle="dotted", color="k", label="Observation point")
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        
sliders = [
        widgets.FloatSlider(value=40, min=20, max=100, step=20, description='Concentration [mg/L]', **param_widgets),
        widgets.FloatSlider(value=10, min=0, max=200, step=1, description='Time [d]', **param_widgets),
        widgets.FloatSlider(value=1, min=0.3, max=10, step=0.1, description='Dispersivity [m]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v2, {
        'Ci': sliders[0],
        't': sliders[1],
        'alpha': sliders[2],
        })

display(ui, out)

Output()

# **2. Analytical solutions for solute transport in porous media with decay**

For this section, we'll see the evolution in time of the concentracion of five different contaminants, each with a different $\lambda$ value.

- 0.0001 day⁻¹ → very slow decay (e.g., nitrate in oxic groundwater, half-life ≈ 19 years)
- 0.001 day⁻¹ → moderate-slow decay (e.g., chlorinated solvents, half-life ≈ 1.9 years)
- 0.01 day⁻¹ → moderate decay (e.g., BTEX compounds biodegradation, half-life ≈ 70 days)
- 0.1 day⁻¹ → relatively fast decay (e.g., pathogens, half-life ≈ 1 week)
- 1.0 day⁻¹ → very fast decay (e.g., labile organic matter, half-life ≈ 0.7 days)

## **2.1. Continuous injection**

### **2.1.1. One-dimensional continuous injection (Grobner & Hofreiter, 1949)**

In this section we visualize the analytical 1D solute transport model with first-order decay in porous media.

For a fixed injection at concentration = 1, it computes how the concentration evolves over time at a given distance for five contaminants, each with a different decay rate (from very slow to very fast).

The user can adjust:
1) Distance from the source
2) Longitudinal dispersivity
3) Retardation factor

The tool then:
1) Computes effective velocity and effective dispersion (accounting for retardation).
2) Calculates beta and gamma parameters used in the analytical solution.
3) Applies the analytical solution including decay.
4) Plots concentration vs. time curves (over two years) for all five decay rates so the user can visually compare attenuation behavior.


The equation that defines the 1D solute transport with decay is the following,

$C(x, t) = \frac{C_{0}}{2}\operatorname{exp}\!\left(\frac{v'}{2D'}\right)\!\left[\operatorname{exp}\!\left( -x \beta \right) \operatorname{erfc}\!\left( \frac{x - \gamma t}{\sqrt{4D't}}\right) + \operatorname{exp}\!\left(x \beta \right) \operatorname{erfc}\!\left(\frac{x + \gamma t}{\sqrt{4D't}}\right)\right]$

where,

$D' = \frac{D}{R}$

$v' = \frac{v}{R}$

$\beta = \sqrt{\frac{v'^{2}}{4D'^{2}} + \frac{\lambda}{D'}}$

$\gamma = \sqrt{v'^{2} + 4\lambda D'}$



In [5]:
def effective_velocity(v: float, R:float) -> float:
    """Calculates effective velocity.

    Parameters
    ----------
    v : float
        velocity [L/T]
    R : float
        Retardation factor [-]

    Returns
    -------
    float
        Effective velocity [L/T]
    """
    
    v_prime = v / R
    return v_prime

def effective_dispersion(v: float, R: float, alpha: float, alpha_tran: float | None = None) -> float | tuple[float, float]:
    """Calculates effective dispersion coefficients considering retardation.
    
    When dispersion coefficient is None, then the function would give a value for both the longitudinal
    and transversal dispersion. When these two latter parameters get None, then dispersion coefficient 
    would be returned. 

    Parameters
    ----------
    v : float
        Groundwater velocity [L/T]
    R : float
        Retardation [-]
    alpha : float
        Dispersivity [L]
    alpha_tran : float | None, optional
        Transversal dispersivity, by default None (when 1D)

    Returns
    -------
    float | tuple[float, float]
        - If only `alpha` and `R` are provided:
            Effective longitudinal dispersion D' [L²/T].
        - If `alpha` and `alpha_tran` are provided:
            Tuple (Effective longitudinal dispersion, Effective transversal dispersion) where each 
            coefficient has been divided by R.
        
    Notes
    -----
    The retardation factor R must be ≥ 1. For R = 1, no retardation occurs and
    effective dispersion equals the original dispersion coefficients.
    """
    
    D_prime = (alpha*v) / R
    if alpha_tran is None:
        return D_prime
    else:
        D_tran_prime = (alpha_tran*v) / R
        return D_prime, D_tran_prime
    

def beta_parameter(v_prime: float, D_prime: float, decay: float) -> float:
    """Calculates the beta parameter.

    Parameters
    ----------
    v_prime : float
        Efective velocity [L/T]
    D_prime : float
        Dispersion coefficient [L²/T]
    decay : float
        Coefficient of radioactive decay [1/T]

    Returns
    -------
    float
        beta parameter [1/L]
    """
        
    Beta = np.sqrt( (v_prime**2/(4*(D_prime**2))) + (decay/D_prime) )
        
    return Beta

def gamma_parameter(v_prime: float, decay: float, D_prime: float) -> float:
    """Calculates gamma parameter. 

    Parameters
    ----------
    v_prime : float
        Effective velocity [L/T]
    decay : float
        Coefficient of radioactive decay [1/T]
    D_prime : float
        Effective dispersion coefficient [L²/T]

    Returns
    -------
    float
        gamma parameter [L/T]
    """
       
    gamma = np.sqrt(v_prime**2 + (4*decay*D_prime))
    
    return gamma

#-------------------------------------------------------#
# 1D continuous injection equation (with decay)         #
#-------------------------------------------------------#
def oneD_decay_continuous_inj(Ci: float, x: float, t: np.ndarray, v: int, alpha: float, decay: float, R: float) -> np.ndarray:
    """Calculates the concentration in a one-dimensional continuous injection framework. 

    Parameters
    ----------
    Ci : float
        Initial concentration [W/V]
    x : float
        distance [L]
    t : float
        time 
    v : int
        velocity [L/T]
    alpha : float
        Dispersivity [L]
    decay : float
        Coefficient of radioactive decay [1/T]
    R : float
        Retardation factor [-]

    Returns
    -------
    np.ndarray
        Concentration [M/V]
    """
    
    v_prime = effective_velocity(v, R) 
    D_prime = effective_dispersion(v, R, alpha) 
    beta = beta_parameter(v_prime, D_prime, decay)
    gamma = gamma_parameter(v_prime, decay, D_prime) 
    
    term1 = (np.exp(-x*beta)) * (sp.special.erfc( (x - (gamma*t)) / (np.sqrt(4*D_prime*t)) ))
    term2 = (np.exp(x*beta)) * (sp.special.erfc( (x + (gamma*t)) / (np.sqrt(4*D_prime*t)) ))
    
    C = ((Ci/2)*np.exp((v_prime*x)/(2*D_prime))) * (term1 + term2)
    
    return C  

In [6]:
def find_parameters_v4(x, alpha, R):        
        decay_values = [0.0001, 0.001, 0.01, 0.1, 1]
        names_list = ["0.0001 [1/d]", "0.001 [1/d]", "0.01 [1/d]", "0.1 [1/d]", "1 [1/d]"]
        
        C = [oneD_decay_continuous_inj(1, x, np.arange(0, 365*2), 1, alpha, decay, R) for decay in decay_values]
 
        #-- Plotting the results
        fig, ax1 = plt.subplots(figsize=(10, 3.5))
                        
        for contaminant in range(len(C)):
                ax1.plot(np.arange(0, 365*2), C[contaminant], label=names_list[contaminant])
        ax1.set_xlim(0, 365*2)
        ax1.set_ylim(0, 1)
        ax1.set_xlabel(r'time [days]')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        
sliders = [
        widgets.FloatSlider(value=100, min=10, max=1000, step=10, description=r'Distance to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=0.3, max=10, step=0.1, description='Dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=1, max=5, step=0.1, description=r'Retardation [-]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v4, {
        'x': sliders[0],
        'alpha': sliders[1],
        'R': sliders[2]
        })

display(ui, out)


Output()

### **2.1.2. Two-dimensional continuous injection (Wilson & Miller, 1978)**

Here, we compute and visualize the analytical 2D solution for continuous solute injection in an infinite porous medium, including first-order decay.

For five contaminants with different decay rates, it calculates how concentration evolves either n time at a fixed observation point (x, y), or in space along the x-axis at a fixed time.

The model incorporates:
1) Retardation
2) Longitudinal and transversal dispersion
3) Radioactive/biodegradation first-order decay
4) 2D spreading using radius, dimensionless time, and the exponential-integral well function approximation

Interactive sliders allow the user to change:
1) Injected mass
2) Distance to observation point (x, y)
3) Time
4) Longitudinal and transversal dispersivities
5) Retardation factor

The equation that defines the 2D continuous injection, including decay, is the following,

$C(x, y, t) = \frac{M/T}{4\pi \phi \left( D'_{L}D'_{T}\right)^{1/2}}\operatorname{exp}\!\left( \frac{x}{B}\right) W\!\left(u, \frac{r}{B}\right)$

where,

$B = \frac{2D'_{L}}{v'}$

$d = 1 + \frac{2B\lambda}{v'}$

$u = \frac{r^{2}}{4dD'_{L}t}$

$r = \sqrt{\left( x^{2} + y^{2} \frac{D'_{L}}{D_{T}} \right)}$

and, when $r/B > 1$, $W\left(u, \frac{r}{B}\right)$ can be approximate as,

$W\left( u, \frac{r}{B} \right) \approx \sqrt{\frac{\pi B}{2r}\operatorname{exp}\!\left( - \frac{r}{B} \right)} \operatorname{erfc}\!\left( \frac{2u - r/B}{2\sqrt{u}} \right)$ 




In [7]:
#-- B parameter 
def B_parameter(D_long_prime: float, v_prime: int) -> float:
    """Calculates the B parameter. 
    
    From Bear & Cheng, 2010 (Eq. 7.4.57)

    Parameters
    ----------
    D_long_prime : float
        Effective longitudinal dispersion coefficient [L²/T]
    v_prime : int
        Effective groundwater velocity [L/T]

    Returns
    -------
    float
        B parameter [L]
    """
    B = (2*D_long_prime)/v_prime
    return B

#-- d parameter
def d_parameter(B: float, v_prime: float, decay: float) -> float:
    """Calculates the d parameter. 
    
    From Bear & Cheng, 2010 (Eq. 7.4.57)

    Parameters
    ----------
    B : float
        B parameter [L]
    v_prime : float
        Effective groundwater velocity [L/T]
    decay : float
        Coefficient of radioactive decay [1/T]

    Returns
    -------
    float
        d parameter [-]
    """
    
    d = 1 + ((2*B*decay)/v_prime)
    return d

#-- radius
def radius(x: float, y: float, D_long_prime: float, D_tran_prime: float, d: float) -> float:
    """Calculates the radius.
    
    From Bear & Cheng, 2010 (Eq. 7.4.57)

    Parameters
    ----------
    x : float
        Position in x axis [L]
    y : float
        Position in y axis [L]
    D_long : float
        Longitudinal dispersion coefficient [L²/T]
    D_tran : float
        Transversal dispersion coefficient [L²/T]

    Returns
    -------
    float
        Radius [L]
    """

    r = np.sqrt( ((x**2) + (y**2*(D_long_prime/D_tran_prime))) * d ) 
    return r

#-- Dimensionless time (u)
def dimensionless_time(r:float, d:float, D_long_prime:float, t:float) -> list:
    """Calculates the dimensionless time (u) and returns a list of its values.
    
    From Bear & Cheng, 2010 (Eq. 7.4.57)

    Parameters
    ----------
    r : float
        Radius [L]
    d : float
        d parameter [-]
    D_long : float
        Longitudinal dispersion coefficient [L²/T]
    t : float
        time

    Returns
    -------
    list
        Dimensionless time (u) [-]
    """

    u = (r**2) / (4*d*D_long_prime*t)
    return u


#-- Solving Well function (W) by means of Exponential Integral Approximation
def well_function(B: float, r: float, u:float):
    """Approximation of the well function (W) value and returns a list of its values.

    Parameters
    ----------
    B : float
        B parameter [L]
    r : float
        radius [L]
    u : float
        dimensionless time [-]

    Returns
    -------
    float
        Well function
    """
    
    W = np.sqrt((np.pi * B)/(2*r)) * np.exp(-(r/B)) * sp.special.erfc(((2*u)-(r/B))/(2*np.sqrt(u)))
    
    return W

In [8]:
#-- Continuous Injection: Solving solute transport in an infinite 2D domain with point source
def twoD_continuous_inj(M: float, T: int, x: float, y: float, t: float, v: float, alpha: float, 
                        alpha_tran: float, R:float, phi: float, decay: float) -> np.ndarray:
    """Calculates the concentration in a two-dimensional continuous injection framework.

    Parameters
    ----------
    M : float
        Total mass flux [M/T]
    T : int
        Injection time
    x : float
        Position in x axis [L]
    y : float
        Position in y axis [L]
    t : float
        time
    v : float
        Groundwater velocity [L/T]
    alpha : float
        Longitudinal dispersivity [L]
    alpha_tran : float
        Transversal dispersivity [L]
    R : float
        Retardation coefficient [-]
    phi : float
        Porosity
    decay : float
        Coefficient of radioactive decay [1/T]

    Returns
    -------
    np.ndarray
        Concentration [M/V]
    """
    
    v_prime = effective_velocity(v, R)
    D_long_prime, D_tran_prime = effective_dispersion(v, R, alpha, alpha_tran)
    B = B_parameter(D_long_prime, v_prime)
    d = d_parameter(B, v_prime, decay)
    r = radius(x, y, D_long_prime, D_tran_prime, d)
    u = dimensionless_time(r, d, D_long_prime, t) 
    W = well_function(B, r, u)
    
    # Bear & Cheng (2010), Eq. 7.4.56
    C = ((M * T) / (4*np.pi*phi*np.sqrt(D_long_prime*D_tran_prime))) * np.exp(x/B) * W
      
    return C

In [9]:
def find_parameters_v5(x, y, M, alpha, alpha_tran, R):   
        decay_values = [0.0001, 0.001, 0.01, 0.1, 1]
        names_list = ["0.0001 [1/d]", "0.001 [1/d]", "0.01 [1/d]", "0.1 [1/d]", "1 [1/d]"]
        
        C = [twoD_continuous_inj(M, 365*2, x, y, np.arange(0, 365*2), 1, alpha, alpha_tran, R, 0.3, decay) for decay in decay_values]
 
        #-- Plotting the results
        fig, ax1 = plt.subplots(figsize=(10, 3.5))
                        
        for contaminant in range(len(C)):
                ax1.plot(np.arange(0, 365*2), C[contaminant], label=names_list[contaminant])
        ax1.set_xlim(0, 365*2)
        # ax1.set_ylim(0, 1500)
        ax1.set_xlabel(r'time [days]')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        

sliders = [
        widgets.FloatSlider(value=75, min=50, max=100, step=5, description=r'Mass injected [g]', **param_widgets),
        widgets.FloatSlider(value=250, min=50, max=2000, step=10, description=r'Distance x to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=0, min=0, max=500, step=10, description=r'Distance y to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=0.2, max=10, step=0.1, description=r'Longitudinal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=0.1, min=0.02, max=1, step=0.001, description=r'Transversal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=1, max=5, step=0.1, description=r'Retardation [-]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v5, {
        'M': sliders[0],
        'x': sliders[1],
        'y': sliders[2],
        'alpha': sliders[3],
        'alpha_tran': sliders[4],
        'R': sliders[5]
        })

display(ui, out)

Output()

In [10]:
def find_parameters_v6(M, t, y, alpha, alpha_tran, R):   
            
        decay_values = [0.0001, 0.001, 0.01, 0.1, 1]
        names_list = ["0.0001 [1/d]", "0.001 [1/d]", "0.01 [1/d]", "0.1 [1/d]", "1 [1/d]"]
        
        C = [twoD_continuous_inj(M, 365*2, np.arange(0, 2000), y, t, 1, alpha, alpha_tran, R, 0.3, decay) for decay in decay_values]
 
        #-- Plotting the results
        fig, (ax1) = plt.subplots(figsize=(10, 5))
        # fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
                        
        #-- Plot 1D
        for contaminant in range(len(C)):
                ax1.plot(np.arange(0, 2000), C[contaminant], label=names_list[contaminant])
        ax1.set_xlim(0, 1000)
        # ax1.set_ylim(0.001, 100000)
        ax1.set_xlabel(r'Distance x [m]')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
                  
sliders = [
        widgets.FloatSlider(value=100, min=1, max=1000, step=1, description=r'time [d]', **param_widgets),
        widgets.FloatSlider(value=75, min=50, max=100, step=5, description=r'Mass injected [g]', **param_widgets),
        widgets.FloatSlider(value=0, min=0, max=500, step=10, description=r'Distance y to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=0.2, max=10, step=0.1, description=r'Longitudinal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=0.1, min=0.02, max=1, step=0.001, description=r'Transversal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=1, max=5, step=0.1, description=r'Retardation [-]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v6, {
        't': sliders[0],
        'M': sliders[1],
        'y': sliders[2],      
        'alpha': sliders[3],
        'alpha_tran': sliders[4],
        'R': sliders[5]
        })

display(ui, out)

Output()

## **2.2. Instantaneous injection**
### **2.2.1. Two-dimensional instantaneous injection (Bear, 1972)**

In this section, we commpute and visualize the analytical 2D instantaneous solute injection solution with first-order decay in porous media. It simulates how concentration evolves over time at a given observation point for five contaminants, each defined by a different decay rate.

With the sliders, the user can interactively modify:
1) Distance to the observation point (x, y)
2) Longitudinal and transversal dispersivities
3) Retardation factor

The tool then automatically recalculates:
1) Effective velocity
2) Effective longitudinal and transverse dispersion
3) Time-dependent concentration curves
  
The following analytical solution defines the 2D instantaneous solute injection with decay,

$C(x, y, t) = \frac{C_{0}A}{4\pi t \sqrt{D'_{L}D'_{T}}}\operatorname{exp}\!\left( - \frac{\left( x-v't \right)^{2}}{4D'_{L}t} - \frac{y^{2}}{4D'_{T}t}\right)\operatorname{exp}\left( - \lambda t \right)$

In [11]:
#-- Instantaneous Injection: Solving solute transport in an infinite 2D domain with point source
def twoD_instantaneous_inj(Ci: float, A: float, x: float, y: float, t: np.ndarray, v: float, alpha: float, 
                           alpha_tran: float, R: float, decay: float) -> np.ndarray:
    """Calculates the concentration in a two-dimensional instantaneous injection framework.

    Parameters
    ----------
    Ci : float
        Initial concentration [M/V]
    A : float
        Contaminated area [L²]
    x : float 
        Position in x axis [L]
    y : float
        Position in y axis [L]
    t : np.ndarray
        Time
    v : float
        Groundwater velocity [L/T]
    alpha : float
        Longitudinal dispersivity [L]
    alpha_tran : float
        Transversal dispersivity [L]
    R : float
        Retardation [-]
    decay : float
        Coefficient of radioactive decay [1/T]

    Returns
    -------
    np.ndarray
        Concentration [M/V]
    """
    
    v_prime = effective_velocity(v, R)
    D_long_prime, D_tran_prime = effective_dispersion(v, R, alpha, alpha_tran)
    
    term1 = (Ci * A) / (4*np.pi*t*np.sqrt(D_long_prime*D_tran_prime))
    term2 = np.exp( - ( ((x-(v_prime*t))**2) / (4*D_long_prime*t) ) - ((y**2)/(4*D_tran_prime*t)) ) 
    term3 = np.exp(-decay*t)
    
    C = term1 * term2 * term3
    C_max = term1 * term3
    return {"C":C, "C_max": C_max}

In [12]:
def find_parameters_v8(x, y, alpha, alpha_tran, R):   
            
        decay_values = [0.0001, 0.001, 0.01, 0.1, 1]
        names_list = ["0.0001 [1/d]", "0.001 [1/d]", "0.01 [1/d]", "0.1 [1/d]", "1 [1/d]"]
               
        results = [twoD_instantaneous_inj(0.864, 500, x, y, np.arange(0, 365*2), 1, alpha, alpha_tran, R, decay) for decay in decay_values]
 
        #-- Plotting the results
        fig, ax1 = plt.subplots(figsize=(10, 3.5))
                        
        for contaminant in range(len(results)):
                ax1.plot(np.arange(0, 365*2), results[contaminant]["C"], label=names_list[contaminant])
        ax1.set_xlim(0, 365*2)
        # ax1.set_ylim(0, 1)
        ax1.set_xlabel(r'time [days]')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        
sliders = [
        widgets.FloatSlider(value=100, min=10, max=1000, step=10, description=r'Distance x to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=0, min=0, max=500, step=10, description=r'Distance y to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=0.2, max=10, step=0.1, description=r'Longitudinal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=0.1, min=0.02, max=1, step=0.001, description=r'Transversal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=1, max=5, step=0.1, description=r'Retardation [-]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v8, {
        'x': sliders[0],
        'y': sliders[1],
        'alpha': sliders[2],
        'alpha_tran': sliders[3],
        'R': sliders[4]
        })

display(ui, out)

Output()

## **3. Principle of Superposition**

This section of the notebook demonstrates the principle of superposition applied to one-dimensional solute transport in porous media. Since the advection–dispersion equation is linear in concentration, individual concentration contributions from different sources or release events can be added to obtain the total solution. The notebook first defines the governing transport equation for continuous injection without decay, using the classical Ogata–Banks analytical solution. It then sets up helper functions for dispersion, the Péclet number, and concentration calculations, along with interactive widgets that allow the user to modify input parameters and visually explore how multiple releases combine.

Two practical examples illustrate how superposition works in groundwater contamination scenarios. In the first example, two overlapping continuous releases—one starting at time zero and another beginning later—are simulated independently and then added to obtain the combined breakthrough curve at an observation point. In the second example, a pulse injection is represented by subtracting two continuous injections, showing how a short-duration spill can be reconstructed using linearity. Interactive plots reveal the timing, peak concentration, and dissipation behavior of each scenario, giving users an intuitive understanding of how linear transport processes respond to multiple or time-varying sources.


### **3.1. Continuous injection**
**ESCENARIO**\
Let's imagine a pollutant source located 100 meters away from observation point A. The pollutant is continuously released in two overlapping stages: from the beginning at a concentration of 800 mg/L, and from day 105 onward an additional release of 450 mg/L occurs simultaneously. The advective velocity is 1 m/d and the dispersion coefficient is 4.42 m²/d.

In [13]:
def find_parameters_v9(FS_C, SS_C):   
       
    #--Parameters
    end_time = 350
    second_stage_time = 105
    second_stage_start = np.zeros(second_stage_time)
    nt = np.arange(second_stage_time, end_time)

    first_stage = oneD_continous_injection(FS_C, 100, 1, np.arange(0, end_time), 4.42, False)
    second_stage = oneD_continous_injection(SS_C, 100, 1, np.arange(0, end_time), 4.42, False)
    second_stage = np.append(second_stage_start, second_stage[1:])
    second_stage = second_stage[:end_time]

    merge_stages = first_stage + second_stage
        
    #--Plotting the results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

    ax1.plot(np.arange(0, end_time), first_stage, c="cornflowerblue", label=f"C({FS_C}, t)")
    ax1.plot(np.arange(0, end_time), second_stage, c="limegreen", label=f"C({SS_C}, t)")
    ax1.set_xlim(0, end_time)
    ax1.set_ylim(0, 1900)
    ax1.set_yticks(np.arange(0, 1900, 200))
    ax1.set_xlabel(r'x')
    ax1.set_ylabel('Concentration')
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    box = ax1.get_position()
    ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])

    ax2.plot(np.arange(0, end_time), merge_stages, c="tomato", label=r"$C_{1}$ + $C_{11}$")
    ax2.set_xlim(0, end_time)
    ax2.set_ylim(0, 1900)
    ax2.set_yticks(np.arange(0, 1900, 200))
    ax2.set_xlabel(r'x')
    ax2.set_ylabel('Concentration')
    ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    box = ax2.get_position()
    ax2.set_position([box.x0, box.y0, box.width * 1.1, box.height])

    fig.tight_layout()
    
    plt.show()
        
sliders = [
        widgets.FloatSlider(value=800, min=100, max=900, step=50, description='First Stage Concentration [mg/L]', **param_widgets),
        widgets.FloatSlider(value=450, min=100, max=900, step=50, description='Second Stage Concentration [mg/L]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v9, {
        'FS_C': sliders[0],
        'SS_C': sliders[1]
        })

display(ui, out)


Output()

### **3.2. Pulse injection**

**ESCENARIO**\
Let's now imagine that, in this second example, there is a punctual spill of the pollutant at a concentration of 800 mg/L, also 100 m from the observation point.

In [14]:
def find_parameters_v10(Ci):   
        #--Parameters
        end_time = 365
        second_stage_time = 105
        second_stage_start = np.zeros(second_stage_time)
        nt = np.arange(second_stage_time, end_time)

        C1 = oneD_continous_injection(Ci, 100, 1, np.arange(0, end_time), 4.42, False)
        C11 = -(oneD_continous_injection(Ci, 100, 1, np.arange(0, end_time), 4.42, False))
        C11 = np.append(second_stage_start, C11[1:])
        C11 = C11[:end_time]
                              
        merge_stages = C1 + C11
        
        maxC11 = np.max(merge_stages)
        merged_lst = list(merge_stages)
        maxC11_pos = merged_lst.index(maxC11)
        noC = []
        for value in range(len(merge_stages)):
                if value > maxC11_pos: 
                        if merge_stages[value] < 1:
                                noC.append(value)                  
                
        #--Plotting the results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

        ax1.plot(np.arange(0, end_time), C1, c="cornflowerblue", label=r"$C_{1}$")
        ax1.plot(np.arange(0, end_time), C11, c="limegreen", label=r"$C_{11}$")
        ax1.hlines(y=0, xmin=0, xmax=end_time, color = "black")
        ax1.set_xlim(0, end_time)
        ax1.set_ylim(-810, 810)
        ax1.set_xlabel(r'time [days]')
        ax1.set_ylabel('Concentration [mg/l]')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])

        ax2.scatter(maxC11_pos, maxC11, c="blue", label=r"Maximum Concentration")
        ax2.plot([0, maxC11_pos], [maxC11, maxC11], c="blue", ls='--')
        ax2.plot([maxC11_pos, maxC11_pos], [maxC11, 0], c="blue", ls='--')
        ax2.scatter(noC[0] + 1, 0.2, c="limegreen", label=r"Concentration ${<}$ 0.2 mg/l")
        ax2.plot(np.arange(0, end_time), merge_stages, c="tomato", label=r"$C_{1}$ + $C_{11}$")
        ax2.set_xlim(0, end_time)
        ax2.set_ylim(0, 810)
        ax2.set_xlabel(r'time [days]')
        ax2.set_ylabel('Concentration [mg/l]')
        ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax2.plot()
        box = ax2.get_position()
        ax2.set_position([box.x0, box.y0, box.width * 1.1, box.height])

        fig.tight_layout()

        plt.show
        
sliders = [
        widgets.FloatSlider(value=400, min=100, max=800, step=50, description='Pollutant Concentration [mg/L]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v10, {
        'Ci': sliders[0]
        })

display(ui, out)

Output()